In [1]:
%load_ext autoreload
%autoreload 2

import boto3
from langchain_community.chat_models import BedrockChat
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (ReActJsonSingleInputOutputParser, ReActSingleInputOutputParser)
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

import sys 
# appending a path - run only once
# sys.path.append('/home/ubuntu/agi-cookbook/agents/patterns') 
print(sys.path)

['/home/ubuntu/agi-cookbook/agents/patterns', '/opt/conda/envs/pytorch/lib/python310.zip', '/opt/conda/envs/pytorch/lib/python3.10', '/opt/conda/envs/pytorch/lib/python3.10/lib-dynload', '', '/opt/conda/envs/pytorch/lib/python3.10/site-packages']


In [5]:
%%writefile utils/react_prompt.py
from typing import Sequence
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

AGENT_PREFIX = """You are an AI assistant. You always stays on topic of the human input and does not diverge from it."""

AGENT_FORMAT_INSTRUCTIONS = """You have access to the following tools:
{tool_descriptions}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

AGENT_QUESTION_PROMPT = """Remember to respond with your knowledge when the question does not correspond to any tool.
The previous conversation is below, where H refers to the human and A refers to the assistant:
{chat_history}

Always append "Final Answer:" when returning the final answer.
Question: {question}"""

AGENT_SUFFIX = """Thought: {agent_scratchpad}"""

def create_agent_prompt(
        tools: Sequence[BaseTool],
        prefix: str = AGENT_PREFIX,
        # context: str = AGENT_CONTEXT,
        format_instructions: str = AGENT_FORMAT_INSTRUCTIONS,
        question_prompt: str = AGENT_QUESTION_PROMPT,
        suffix: str = AGENT_SUFFIX,
    ) -> PromptTemplate:
    
        human_prompt = PromptTemplate(
            input_variables=[
                "chat_history", 
                "question",
                "agent_scratchpad"
            ],
            partial_variables={
                "tool_names": ", ".join([tool.name for tool in tools]),
                "tool_descriptions": "\n".join(
                    [f"    {tool.name}: {tool.description}" for tool in tools]
                )
            },
            template='\n'.join(
                [
                    prefix,
                    format_instructions,
                    # context,
                    question_prompt,
                    suffix
                ]
            )
        )
        human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)
        return ChatPromptTemplate.from_messages([human_message_prompt])

Overwriting utils/react_prompt.py


In [1]:
from utils.react_prompt import create_agent_prompt
from utils.tools import tools

# tools = tools[:2] 
print(f"len(tools): {len(tools)}")

prompt = create_agent_prompt(tools)
print(prompt.messages[0].prompt.template)

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "max_tokens": 4096, 
        "temperature": 0.5,
    },
    streaming=False,
    region_name="us-west-2",
    # region_name="us-east-1",
)

len(tools): 9
You are an AI assistant. You always stays on topic of the human input and does not diverge from it.
You have access to the following tools:
{tool_descriptions}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Remember to respond with your knowledge when the question does not correspond to any tool.
The previous conversation is below, where H refers to the human and A refers to the assistant:
{chat_history}

Always append "Final Answer:" when returning the final answer.
Question: {question}
Thought: {agent_scratchpad}


NameError: name 'BedrockChat' is not defined

In [7]:
import re
from typing import List, Union
from langchain.agents import  AgentOutputParser
from langchain.schema import AgentAction, AgentFinish

class CustomOutputParser(AgentOutputParser):    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # print("llm_output",llm_output)
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [8]:
agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        # "agent_scratchpad": lambda x: x["intermediate_steps"],
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm.bind(stop=["\nObservation"])
    # | ReActJsonSingleInputOutputParser()
    # | ReActSingleInputOutputParser()
    | CustomOutputParser()
)

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True, 
    input_key="question",
    output_key="output",
    ai_prefix="A",
    human_prefix="H",
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
)

In [9]:
%%time
result = agent_executor.invoke({
    "question": "what are Einstein's top 3 research papers ?",
})



> Entering new AgentExecutor chain...


Thought: To find Einstein's top 3 research papers, I should search for information on his most influential or highly cited scientific publications.
Action: search-google-scholar
Action Input: "einstein most influential research papers"Title: Why Einstein became famous in America
Authors: 
Summary: M Missner - Social studies of science, 1985 - journals.sagepub.com
Total-Citations: 87

Title: E= Einstein: His life, his thought, and his influence on our culture
Authors: M Bartusiak
Summary: D Goldsmith, M Bartusiak - 2008 - books.google.com
Total-Citations: 20

Title: Einstein
Authors: 
Summary: T Ryckman - 2017 - taylorfrancis.com
Total-Citations: 35

Title: Albert Einstein
Authors: 
Summary: A Einstein - Quantum Algebra and Symmetry, 2005 - researchgate.net
Total-Citations: 30

Title: Albert Einstein on his seventieth birthday
Authors: 
Summary: RA Millikan - Reviews of Modern Physics, 1949 - APS
Total-Citations: 47

Title: Albert Einstein as a philosopher of science
Authors: DA Howard


In [10]:
print(result["output"])

Based on their scientific impact and number of citations, Einstein's top 3 research papers are considered to be:

1. The "Annus Mirabilis" papers of 1905 
2. "The Field Equations of Gravitation" (1915) on general relativity
3. "On the Electrodynamics of Moving Bodies" (1905) on special relativity


In [23]:
print(result["output"])

Albert Einstein's top 3 most influential research papers are:

1. His 1905 paper on the special theory of relativity
2. His 1905 paper on the photoelectric effect, for which he won the Nobel Prize 
3. His 1905 paper providing a theoretical explanation for Brownian motion


In [12]:
%%time
result = agent_executor.invoke({
    "question": "show the proof of e equals mc2"
})



> Entering new AgentExecutor chain...
Thought: To show the proof of E = mc^2, I will need to search for resources that provide the derivation and explanation of this famous equation from Einstein's theory of relativity.

Action: search-wikipedia
Action Input: Einstein mass-energy equivalence
Page: Mass–energy equivalence
Summary: In physics, mass–energy equivalence is the relationship between mass and energy in a system's rest frame, where the two quantities differ only by a multiplicative constant and the units of measurement. The principle is described by the physicist Albert Einstein's formula: E=mc2{\displaystyle E=mc^{2}}. In a reference frame where the system is moving, its relativistic energy and relativistic mass (instead of rest mass) obey the same formula.
The formula defines the energy E of a particle in its rest frame as the product of mass (m) with the speed of light squared (c2). Because the speed of light is a large number in everyday units (approximately 300000 km/s o

In [13]:
print(result["output"])

The proof of Einstein's famous equation E = mc^2 comes from his theory of special relativity. The key principles used are:

1) The laws of physics are the same in all inertial (non-accelerating) reference frames. 
2) The speed of light in a vacuum is constant, independent of the motion of the source or observer.

From these postulates, Einstein derived the relativistic equations for energy and momentum. Applying the Lorentz transformations between different reference frames, and setting the momentum equal to zero for a particle at rest, one can show that the mass (m) and energy (E) of a particle are related by:

E = mc^2

Where c is the speed of light in a vacuum. This famous equation expresses the equivalence of mass and energy. The conversion factor c^2 (nearly 9 x 10^16 m^2/s^2) is an enormously large number. This means that even a small amount of mass is associated with a tremendous amount of energy.

For example, the nuclear reactions that power the sun convert about 0.7% of the m

In [14]:
%%time
result = agent_executor.invoke({
    "question": "what are the most influential research contributions in large language models ?"
})



> Entering new AgentExecutor chain...
Thought: To answer this question about the most influential research contributions in large language models, I should search for academic papers and articles on this topic using search engines like Google Scholar and arXiv.

Action: search-google-scholar
Action Input: influential research papers large language models
Title: Tracing the Influence of Large Language Models across the Most Impactful Scientific Works
Authors: DM Petroșanu,A Pîrjan,A Tăbușcă
Summary: DM Petroșanu, A Pîrjan, A Tăbușcă - Electronics, 2023 - mdpi.com
Total-Citations: 2

Title: A survey on evaluation of large language models
Authors: X Wang,J Wang,Y Wu,L Yang
Summary: Y Chang, X Wang, J Wang, Y Wu, L Yang… - ACM Transactions on …, 2023 - dl.acm.org
Total-Citations: 409

Title: Susceptibility to influence of large language models
Authors: LD Griffin,B Kleinberg,M Mozes,KT Mai
Summary: LD Griffin, B Kleinberg, M Mozes, KT Mai, M Vau… - arXiv preprint arXiv …, 2023 - arxiv.or

In [15]:
print(result["output"]) # need to do parallel search

Some of the most influential research contributions in large language models include:

1. Analyses of the capabilities, limitations, and risks of models like GPT-3 (e.g. "On the dangers of stochastic parrots" by Bender et al.)

2. Surveys and reviews on architectures, applications, and challenges (e.g. "Large language models: A survey" by Minaee et al.) 

3. Evaluations of model performance on tasks like code generation (e.g. "A systematic evaluation of large language models of code" by Xu et al.)

4. Studies on the societal impacts across domains like science and social media (e.g. "Tracing the Influence of Large Language Models" by Petroșanu et al.)

5. Work on evaluation metrics and frameworks for assessing model outputs (e.g. "A survey on evaluation of large language models" by Wang et al.)


In [5]:
%%time
result = agent_executor.invoke({
    "question": "what is state of the art text to sql model that can used in production applications ?"
})



> Entering new AgentExecutor chain...


Thought: To find a state-of-the-art text-to-SQL model that can be used in production applications, I will need to search for recent research papers and articles on this topic. Google Scholar and arXiv would be good sources to look for relevant information.

Action: search-google-scholar
Action Input: text to sql model production
Title: Chinese Text-to-SQL model for industrial production
Authors: 
Summary: J LYU, X WANG, G CHEN, H ZHANG… - Journal of Computer …, 2022 - joca.cn
Total-Citations: 

Title: A survey on deep learning approaches for text-to-SQL
Authors: G Katsogiannis-Meimarakis,G Koutrika
Summary: G Katsogiannis-Meimarakis, G Koutrika - The VLDB Journal, 2023 - Springer
Total-Citations: 44

Title: Grammar-based neural text-to-sql generation
Authors: K Lin,B Bogin,M Neumann,J Berant
Summary: K Lin, B Bogin, M Neumann, J Berant… - arXiv preprint arXiv …, 2019 - arxiv.org
Total-Citations: 50

Title: Text-to-SQL generation for question answering on electronic medical records
Auth

In [6]:
print(result["output"])

Based on recent research papers, some state-of-the-art text-to-SQL models that could potentially be used in production applications include Grammar-based neural text-to-SQL generation, Robust text-to-sql generation with execution-guided decoding, Decoupled dialogue modeling and semantic parsing for multi-turn text-to-SQL, Natural SQL, and the models evaluated in the benchmarking work by Zhang et al. (2024). The specific choice would depend on factors like the application domain, query complexity, available data, and desired trade-offs.


In [7]:
# the answers are not factually correct -> there's a need to use multiple tools at a time and reason through

### TODO: use parallel tools

Something like this - BaseMultiActionAgent

- https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/openai_functions_multi_agent/base.py
- https://twitter.com/LangChainAI/status/1644387089851781120
- https://github.com/langchain-ai/langchain/blob/master/cookbook/custom_multi_action_agent.ipynb


write prompts to test the below output parsers:
- ReActJsonSingleInputOutputParser()
- ReActSingleInputOutputParser()

In [ ]:
# https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/json.py#L128

def parse_json_markdown(
    json_string: str, *, parser: Callable[[str], Any] = parse_partial_json
) -> dict:
    """
    Parse a JSON string from a Markdown string.

    Args:
        json_string: The Markdown string.

    Returns:
        The parsed JSON object as a Python dictionary.
    """
    try:
        return _parse_json(json_string, parser=parser)
    except json.JSONDecodeError:
        # Try to find JSON string within triple backticks
        match = re.search(r"```(json)?(.*)", json_string, re.DOTALL)

        # If no match found, assume the entire string is a JSON string
        if match is None:
            json_str = json_string
        else:
            # If match found, use the content within the backticks
            json_str = match.group(2)
    return _parse_json(json_str, parser=parser)

    def _parse_json(
    json_str: str, *, parser: Callable[[str], Any] = parse_partial_json
) -> dict:
    # Strip whitespace and newlines from the start and end
    json_str = json_str.strip().strip("`")

    # handle newlines and other special characters inside the returned value
    json_str = _custom_parser(json_str)

    # Parse the JSON string into a Python dictionary
    return parser(json_str)